<a href="https://colab.research.google.com/github/mralamdari/Robotics-in-Python/blob/main/Robotics_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import scipy
import sympy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
#Denavite Hartnzberg
def DH(a, alpha_degree, d, theta_degree):
  theta = np.deg2rad(theta_degree)
  alpha = np.deg2rad(alpha_degree)
  dh_table = np.array([[np.cos(theta), -np.cos(alpha)*np.sin(theta), np.sin(alpha)*np.sin(theta), a*np.cos(theta)],
                      [np.sin(theta), np.cos(alpha)*np.cos(theta), -np.sin(alpha)*np.cos(theta), a*np.sin(theta)],
                      [0 ,np.sin(alpha), np.cos(alpha), d],
                      [0, 0, 0, 1]])
  return np.float16(dh_table)

In [5]:
a1 =DH(0, -90, -999, 0)
a2 =DH(0, -90, -888, 90)
a3 =DH(0, 0, -777, 0)

s=np.dot(a1, a2)
s

array([[   0.,    0.,   -1.,    0.],
       [   0.,   -1.,    0., -888.],
       [  -1.,    0.,    0., -999.],
       [   0.,    0.,    0.,    1.]], dtype=float16)

In [6]:
np.dot(s, a3)

array([[   0.,    0.,   -1.,  777.],
       [   0.,   -1.,    0., -888.],
       [  -1.,    0.,    0., -999.],
       [   0.,    0.,    0.,    1.]], dtype=float16)

#Forwarsd Synematic

0 ===> Scara

1 ===> manipulator

2 ===> cyliendrical

3 ===> stanford

In [7]:
def forward_scara(a1, a2, d3, d4, theta_1, theta_2, theta_4):
  A01 = DH(a1, 0, 0, theta_1)
  A12 = DH(a2, 180, 0, theta_2)
  A23 = DH(0, 0, d3, 0)
  A34 = DH(0, 0, d4, theta_4)

  T40 = np.matmul(np.matmul(A01, A12), np.matmul(A23,A34))
  return np.float16(T40)

In [8]:
def forward_manipulator(a1, a2, theta_1, theta_2):
  A01 = DH(a1, 0, 0, theta_1)
  A12 = DH(a2, 0, 0, theta_2)

  T20 = np.matmul(A01, A12)
  return np.float16(T20)

In [9]:
def forward_cylindrical(alpha_2, d1, d2, d3, theta_1):
  A01 = DH(0, 0, d1, theta_1)
  A12 = DH(0, alpha_2, d2, 0)
  A23 = DH(0, 0, d3, 0)

  T30 = np.matmul(np.matmul(A01, A12), A23)
  return np.float16(T30)

In [10]:
def forward_stanford(d2, d3, theta_1, theta_2):
  A01 = DH(0, -90, 0, theta_1)
  A12 = DH(0, 90, d2, theta_2)
  A23 = DH(0, 0, d3, 0)

  T30 = np.matmul(np.matmul(A01, A12), A23)
  return np.float16(T30)

In [11]:
def wrist(alpha_4, alpha_5, d6, theta_4, theta_5, theta_6):
  A34 = DH(0, -90, 0, theta_4)
  A45 = DH(0, 90, 0, theta_5)
  A56 = DH(0, 0, d6, theta_6)

  T63 = np.matmul(np.matmul(A34, A45), A56)
  return np.float16(T63)

In [12]:
def forward_robotic(point, num=0, wist=False, a1=0, a2=0, a3=0, a4=0, a5=0, a6=0, alpha_1=0, alpha_2=0, alpha_3=0, alpha_4=0, alpha_5=0, alpha_6=0,  d1=0, d2=0, d3=0, d4=0, d5=0, d6=0, theta_1=0, theta_2=0, theta_3=0, theta_4=0, theta_5=0, theta_6=0):
  
  
  point=np.array(point).reshape(-1, 1)
  if num == 0:
    T0_3 = forward_scara(a1, a2, d3, d4, theta_1, theta_2, theta_4)
  elif num == 1:  
    T0_3 = forward_manipulator(a1, a2, theta_1, theta_2)
  elif num == 2:  
    T0_3 =  forward_cylindrical(alpha_2, d1, d2, d3, theta_1)
  else:  
    T0_3 = forward_stanford(d2, d3, theta_1, theta_2)

  if wist:
    T3_6 = wrist(alpha_4, alpha_5, d6, theta_4, theta_5, theta_6)
    T = np.matmul(T0_3, T3_6)
  else:
    T = T0_3

  R = T[:-1, :-1]
  d = T[:-1, -1].reshape(-1, 1)  
  res = np.matmul(R, point) + d

  return res.T[0]

In [13]:
forward_robotic([12,23,53],
                 num=3,
                 wist=True,
                 a1=2,
                 a2=2,
                 d1=20,
                 d2=40,
                 d3=30,
                 d4=10 ,
                 theta_1=60,
                 theta_2=-180,
                 theta_4=25)

array([-57.68408203,  31.95996094, -83.        ])

#Tabdils and Rotations
R_x, R_y, R_z

T_x, T_y, T_z

In [14]:
def T_x(a):
  TX = np.array([[1, 0, 0, a],
                 [0, 1, 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
  return TX

In [15]:
def T_y(b):
  TY = np.array([[1, 0, 0, 0],
                 [0, 1, 0, b],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
  return TY

In [16]:
def T_z(c):
  TZ = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 1, c],
                 [0, 0, 0, 1]])
  return TZ